In [11]:
import pandas as pd
import os

# Step 1: Read the C019 Orginal RFQ File
Quote_Date = 20250203
RFQ_NO = 6001003359
Base_Path = r"Z:\業務部\業務一課\G-報價\1. 外銷\C01900 Reyher\2025"
RFQ_File_Name = f"_RFQ_6001003359_20250204_081628.xlsx"
RFQ_Fie_Path = os.path.join(Base_Path,f"{Quote_Date} {RFQ_NO}",RFQ_File_Name)
DF_Original_RFQ = pd.read_excel(RFQ_Fie_Path, sheet_name="RFQ positions", skiprows=9)

# Step 2: Extract text before the second '.' for product new product series
RFQ_col_name = "Material"
if RFQ_col_name in DF_Original_RFQ.columns:
    DF_Original_RFQ["original_category"] = DF_Original_RFQ[RFQ_col_name].astype(str).apply(lambda x: ".".join(x.split(".")[:2]))

# Step 3: Get the previous product series
Category_Compare_File_Path = r"Z:\業務部\業務一課\G-報價\1. 外銷\C01900 Reyher\2025\C019 新舊產品代號對照表_Caysie.xlsx"
# DF_Item_Series = pd.read_excel(Category_Compare_File_Path, sheet_name="Total", dtype={"新產品代號": str})
DF_Item_Series = pd.read_excel(Category_Compare_File_Path, sheet_name=None, dtype=str)

# Step 4: replace original_category with 舊產品代號 and names as old_category
for num, Item_Category in DF_Original_RFQ["original_category"].items():
    for idx, Original_Category in DF_Item_Series["新產品代號"].items():  
        if Item_Category == Original_Category:
            DF_Original_RFQ.at[num, "old_category"] = DF_Item_Series.at[idx, "舊產品代號"]

# Step 5: Get the previous item code for DB running
if col_name in DF_Original_RFQ.columns:
    DF_Original_RFQ["material_suffix"] = DF_Original_RFQ[col_name].astype(str).apply(lambda x: ".".join(x.split(".")[2:]))
    DF_Original_RFQ["previous_item_code"] = DF_Original_RFQ["old_category"].astype(str) + "." + DF_Original_RFQ["material_suffix"]
    
    # Reorder columns to place "previous_item_code" next to "Material"
    cols = DF_Original_RFQ.columns.tolist()
    cols.insert(cols.index("Material") + 1, cols.pop(cols.index("previous_item_code")))
    DF_Original_RFQ = DF_Original_RFQ[cols]





,新產品代號,舊產品代號
0,007504.00106,07504.016
1,007504.00107,07504.019
2,007504.00108,07504.301
3,015480.00100,07504.014
4,015481.00104,07504.015
5,015482.00104,07504.016


In [24]:
import sqlite3

# Connect to DB
DB_File_Path = r"Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"
with sqlite3.connect(DB_File_Path) as connect : 
    query = "SELECT PRODUCT_CODE FROM CUSTOMER_PRODUCT_SUMMARY WHERE CUSTOMER_CODE = 'C01900'"
    DF_DB = pd.read_sql_query(query, connect)

In [66]:
# Ensure PRODUCT_CODE is a set for fast lookups
product_code_set = set(DF_DB["PRODUCT_CODE"].astype(str))

# Check existence of original code first than the previous code, to get searching code for SP TOOL BOX running
def check_existence(DF_Codes):
    DF_Codes["SEARCHING_CODE"] = 0  

    for num, row in DF_Codes.iterrows():
        material = str(row["Material"])
        final_material = str(row["previous_item_code"])
        
        if material in product_code_set:
            DF_Codes.loc[num, "SEARCHING_CODE"] = material
        elif final_material in product_code_set:
            DF_Codes.loc[num, "SEARCHING_CODE"] = final_material
        else: 
            DF_Codes.loc[num, "SEARCHING_CODE"] = "Not found"

    return DF_Codes

# # Apply function and get final DF
DF_Final = check_existence(DF_Original_RFQ)

# Save the result to a new Excel file (overwrite existing file)
output_file = os.path.join(Base_Path,f"{Quote_Date} {RFQ_NO}", f"Updated_RFQ.xlsx")

selected_columns = ["Item", "Material", "previous_item_code", "Description", 
                    "RFQ Quantity", "Package Qty", "SEARCHING_CODE"]

if os.path.exists(output_file):
    os.remove(output_file)

DF_Final[selected_columns].to_excel(output_file, index=False)

# print("File successfully saved:", output_file)

C:\Users\sales6\AppData\Local\Temp\ipykernel_19208\4007292447.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '00965.301.0040.008' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  DF_Codes.loc[num, "SEARCHING_CODE"] = final_material
